In [7]:
import os
import json
import numpy as np
import math
from tqdm import tqdm as tq
from PIL import Image
from torchvision import transforms
import torch

In [8]:
img_path = '/home/rakshith/Datasets/Task03_Liver/imagesTr/'
label_path = '/home/rakshith/Datasets/Task03_Liver/labelsTr/'

image_list = os.listdir(img_path)
label_list = os.listdir(label_path)
train_list = [image_list[int(i)].split('.nii')[0] for i in range(int(0.8*len(image_list)))]
valid_list = [image_list[int(i)].split('.nii')[0] for i in range(int(0.8*len(image_list)),len(image_list))]

data_dict = {
    "train": train_list,
    "valid": valid_list
}
with open('data_split.json', 'w') as f:
    json.dump(data_dict,f)
# for img_name in image_list:
#     img = nib.load(img_path+img_name)
#     img = img.get_fdata()
#     label = nib.load(label_path+img_name)
#     label = label.get_fdata()
#     print(img.shape)
#     print(label.shape)
#     np.save('/home/rakshith/Datasets/Task03_Liver/np_array/images_tr/'+img_name, img)
#     np.save('/home/rakshith/Datasets/Task03_Liver/np_array/labels_tr/'+img_name, label)
#     break

In [12]:
print(len(train_list))
print(len(valid_list))

209
53


In [11]:
img_path = '/home/rakshith/Datasets/Task03_Liver/np_array_slices/images_tr/'
img_list = os.listdir(img_path)

train_files = [file for file in img_list if file.split('_')[0]+'_'+file.split('_')[1] in train_list]
valid_files = [file for file in img_list if file.split('_')[0]+'_'+file.split('_')[1] in valid_list]

data_dict = {
    "train": train_files,
    "valid": valid_files
}
with open('train_valid_split.json', 'w') as f:
    json.dump(data_dict,f)


In [1]:


img_path = '/home/rakshith/Datasets/Task03_Liver/np_array_slices/images_tr/'
img_list = os.listdir(img_path)
min_list, max_list = [], []
for img_name in tq(img_list):
    # print(f'Count: {i} Total: {len(img_list)}')
    img = np.load(img_path+img_name)
    min_list.append(np.min(img))
    max_list.append(np.max(img))
print(np.min(min_list))
print(np.max(max_list))




100%|██████████| 19163/19163 [01:43<00:00, 184.38it/s]


-2048.0
27572.0


  0%|          | 0/19163 [00:00<?, ?it/s]


UnidentifiedImageError: cannot identify image file '/home/rakshith/Datasets/Task03_Liver/np_array_slices/images_tr/liver_103_623.npy'

In [6]:
sum = 0
numerator = 0
img_path = '/home/rakshith/Datasets/Task03_Liver/np_array_slices/images_tr/'
img_list = os.listdir(img_path)

for i in tq(range(len(img_list))):
    image_sample = np.load(img_path+img_list[i])
    image_sample[image_sample>1000] = 1000
    image_sample[image_sample<-1000] = -1000
    image_func = transforms.ToTensor()
    image_sample = image_func(image_sample).cuda()
    sum += torch.sum(image_sample)
    # print(image_sample.shape)

mean = sum/(len(img_list)*1024*1024)

for i in tq(range(len(img_list))):
    image_sample = np.load(img_path+img_list[i])
    image_sample[image_sample>1000] = 1000
    image_sample[image_sample<-1000] = -1000
    image_func = transforms.ToTensor()
    image_sample = image_func(image_sample).cuda()
    numerator += torch.sum((image_sample - mean)**2)

std = math.sqrt(numerator/(len(img_list)*1024*1024))

print('Mean:',mean)
print('SD:',std)

100%|██████████| 19163/19163 [06:57<00:00, 45.91it/s] 

Mean: tensor(-124.8561, device='cuda:0', dtype=torch.float64)
SD: 312.06272161597764


In [1]:
import torch
from self_attn_sumnet import FullAttenSUMNet

net = FullAttenSUMNet(1,2).cuda()
inp = torch.ones(2,1,256,256).cuda()
out = net(inp)

RuntimeError: CUDA out of memory. Tried to allocate 288.00 MiB (GPU 0; 15.78 GiB total capacity; 14.18 GiB already allocated; 188.50 MiB free; 14.29 GiB reserved in total by PyTorch)